### Installing Libraries

In [1]:
!pip install -q cassio datasets langchain openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 839.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 82.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [2]:
pip install faiss-cpu


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 42.7 MB/s eta 0:00:00


In [3]:
import faiss


### Import the packages

In [4]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from datasets import load_dataset
import cassio

In [5]:

!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.2 MB/s eta 0:00:00


In [6]:
from PyPDF2 import PdfReader

### Setup all the tokens and api keys


In [7]:
import os

In [8]:
ASTRA_DB_APPLICATION_TOKEN = "AstraCS:YiBvylgvMOanmFjYxjTqBlAm:2265f41ae0c862fc8f8560b88e37890389c461b3b96b3bf9aa778403c0ed44b4"
ASTRA_DB_ID = "eb9700c2-fc53-43ca-988f-ad6d4df16fe9" # enter your Database ID
OPENAI_API_KEY ="sk-proj-6MJfPIj5KDBPBxzCuxBBT3BlbkFJmL198mgSzQLxuVh1yB3y" # enter your OpenAI key

### provide the path of  pdf file/files.


In [9]:
pdfreader = PdfReader('/content/tourist_attractions_combined.pdf')

In [10]:
from typing_extensions import Concatenate
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [ ]:
raw_text

'CITY==  Hyderabad  \n  \nURL: https://en.wikipedia.org/wiki/List_of_tourist_attractions_in_Hyderabad  \n  \n  \nCategory: Historical monuments  \n- Charminar ± a major landmark of Hyderabad with four graceful minarets located in the old city. It was \nbuilt by Muhammed Quli Qutb Shah as a memorial for plague victims. Charminar, on most occasions, is \nused to represent the city and the state and is hailed as a unique Deccan monument. It is in the midst of \nCharkaman which are four archways to roads leading in all four directions from the Charminar \nmonument.[9]  \n- Mecca Masjid ±  A symbolic mosque, Mecca Masjid was built 400 years ago during the reign of Sultan \nMuhammad Qutub Shah, the 6th Qutub Shahi Sultan of Hyderabad. The three-arched facade has been \ncarved from a single piece of granite, which took five years to quarry. More than 8,000 workers were \nemployed to build the mosque. Muhammed Qutub Shah personally laid the foundation stone.  \n  \nCategory: Palaces  \n- Falak

In [ ]:
len(raw_text)

518144

### Initialize the connection to your database


In [9]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

ERROR:cassandra.connection:Closing connection <LibevConnection(133023894163648) eb9700c2-fc53-43ca-988f-ad6d4df16fe9-us-east1.db.astra.datastax.com:29042:05c9a152-b1cf-4ea1-82d7-bda86f884677> due to protocol error: Error from server: code=000a [Protocol error] message="Beta version of the protocol used (5/v5-beta), but USE_BETA flag is unset"


### Create the LangChain embedding and LLM objects

In [ ]:
llm = OpenAI(openai_api_key=OPENAI_API_KEY)
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


### Create your LangChain vector store

In [ ]:
astra_vector_store = Cassandra(
    embedding=embedding,
    table_name="qa_mini_demo",
    session=None,
    keyspace=None,
)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
texts[0]

'CITY==  Hyderabad  \n  \nURL: https://en.wikipedia.org/wiki/List_of_tourist_attractions_in_Hyderabad  \n  \n  \nCategory: Historical monuments  \n- Charminar ± a major landmark of Hyderabad with four graceful minarets located in the old city. It was \nbuilt by Muhammed Quli Qutb Shah as a memorial for plague victims. Charminar, on most occasions, is \nused to represent the city and the state and is hailed as a unique Deccan monument. It is in the midst of \nCharkaman which are four archways to roads leading in all four directions from the Charminar \nmonument.[9]  \n- Mecca Masjid ±  A symbolic mosque, Mecca Masjid was built 400 years ago during the reign of Sultan \nMuhammad Qutub Shah, the 6th Qutub Shahi Sultan of Hyderabad. The three-arched facade has been'

In [ ]:
texts[1]

"- Mecca Masjid ±  A symbolic mosque, Mecca Masjid was built 400 years ago during the reign of Sultan \nMuhammad Qutub Shah, the 6th Qutub Shahi Sultan of Hyderabad. The three-arched facade has been \ncarved from a single piece of granite, which took five years to quarry. More than 8,000 workers were \nemployed to build the mosque. Muhammed Qutub Shah personally laid the foundation stone.  \n  \nCategory: Palaces  \n- Falaknuma Palace ± built by an Italian architect through one of the Paigah nobles, Nawab Viqar \nal-Umra', complete with Italian marble, Louis XIV-style decor, a Mughal ambience, Italian marble \nstaircases and ornate fountains. It has now been undertaken by Taj group to develop this as a heritage"

In [ ]:
texts[2]

"staircases and ornate fountains. It has now been undertaken by Taj group to develop this as a heritage \nhotel. It had taken 9 years to build the palace, from 1884 to 1893. The library in the Palace is said to be \nas majestic as the one in the Windsor Castle. One of the main attractions of the palace is the 100 seater \ntable that is 108 feet long  \n- Chowmahalla Palace ± It was the seat of Asaf Jahi dynasty, where the Nizam entertained his official \nguests and royal visitors. Initiated in 1750 by Nizam Salabat Jung and designed along the lines of the \nShah's palace in Isfahan, this actually consists of a group of palaces each used as a Durbar Hall. It has \nnow been aesthetically renovated and is also a venue for conferences.[12]"

In [ ]:
len(texts)

848

### Load the dataset into the vector store



In [ ]:
astra_vector_store.add_texts(texts)
print("Inserted %i headlines." % len(texts))
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 848 headlines.


### ------------------------------------------- Travel ChatBot ---------------------------------------



In [ ]:
import pandas as pd
# Flag to check if it's the first question
first_question = True

In [ ]:
while True:
    if first_question:
        query_text = input("\nEnter your question (or type 'quit' to exit): ").strip()
    else:
        query_text = input("\nWhat's your next question (or type 'quit' to exit): ").strip()

    if query_text.lower() == "quit":
        break

    if query_text == "":
        continue

    first_question = False

    print("\nQUESTION: \"%s\"" % query_text)

    answer = astra_vector_index.query(query_text, llm=llm).strip()
    print("ANSWER: \"%s\"\n" % answer)
    print("FIRST 4 DOCUMENTS BY RELEVANCE:")
    for doc, score in astra_vector_store.similarity_search_with_score(query_text, k=4):
      print("    [%0.4f] \"%s ...\"" % (score, doc.page_content[:1000]))
# a day trip to Hyderabad to visit Temples,lakes etc


Enter your question (or type 'quit' to exit): Give some places related to nature, museums and beaches in chennai

QUESTION: "Give some places related to nature, museums and beaches in chennai"


ANSWER: "Some places in Chennai related to nature are Aringar Anna Zoological Park in Vandalur, Valluvar Kottam, and Chennai viewed from St. Thomas Mount. Museums in Chennai include the Government Museum at Egmore and Chettinad Palace. Beaches in Chennai include Marina Beach, Elliot's Beach, and Blue Flag Beach."

FIRST 4 DOCUMENTS BY RELEVANCE:


    [0.9307] "-   
  
War Memorial  
  
-   
  
Schmidt memorial at Elliot's beach  
  
-   
  
Government Museum at Egmore  
  
-     
Valluvar Kottam  
  
-   
  
Aringar Anna Zoolological Park in Vandalur  
  
-   
  
The Victoria Public Hall  
  
  
Category: Gallery  
-   
  
Santhome Basilica  
  
-   
  
Interior of Santhome Basilica  
  
-   
  
Chettinad Palace  
  
-   
  
Chennai viewed from St. Thomas Mount  
  
-   
  
War Memorial  
  
-   
  
Schmidt memorial at Elliot's beach  
  
-   
  
Government Museum at Egmore  
  
-   
  
Valluvar Kottam  
  
-   
  
Aringar Anna Zoolological Park in Vandalur  
  
-   
  
The Victoria Public Hall  
  
  
Category: See also  - India portal  
  
Category: References  
- Places to visit in Chennai  
- Oneday Trip Around Chennai ..."
    [0.9307] "-   
  
War Memorial  
  
-   
  
Schmidt memorial at Elliot's beach  
  
-   
  
Government Museum at Egmore  
  
-     
Valluvar Kottam  
  
-   
  
Aringar Anna Zoolological Park in Vand

### CODE 2

In [ ]:
# Read PDF and extract text


In [ ]:
from PyPDF2 import PdfReader

pdf_path = '/content/sodapdf-converted.pdf'
pdf_reader = PdfReader(pdf_path)
raw_text = ''

for page in pdf_reader.pages:
    raw_text += page.extract_text()


In [ ]:
# Split text into chunks


In [ ]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)

text_chunks = text_splitter.split_text(raw_text)

In [ ]:
text_chunks[0]

"CITY==  Chennai  \n  \nURL: https://en.wikipedia.org/wiki/List_of_tourist_attractions_in_Chennai  \n  \n  \nCategory: Tourist arrival statistics  \n- Marina Beach is a 13-km-long urban beach along the Bay of Bengal that runs from Fort St. George in \nthe north up to Foreshore Estate in the south. It is India's longest beach and one of the world's longest \nbeaches and attracts around 50,000 visitors during weekends. Attractions at the Marina include the \nChennai Lighthouse, MGR Memorial, Anna Memorial and Jayalalitha Memorial. The Marina's 6 km \npromenade includes statues of several historical figures including Mahatma Gandhi, Annie Besant, \nRobert Caldwell, Thiruvalluvar, Bharathiyar and Kamrajar.MGR Samadhi in Marina Beach"

In [ ]:
from langchain_community.vectorstores import FAISS



In [ ]:
# Initialize OpenAI embeddings


In [ ]:
from langchain.embeddings import OpenAIEmbeddings
import faiss

OPENAI_API_KEY = "sk-proj-6MJfPIj5KDBPBxzCuxBBT3BlbkFJmL198mgSzQLxuVh1yB3y"
embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
# Assuming you have code to create embeddings for each chunk of text and store them in a Faiss index


In [ ]:
import numpy as np

# Initialize Faiss index
d = 768  # Embedding dimension
index = faiss.IndexFlatL2(d)  # Assuming L2 distance metric
vector_store = {}

In [ ]:
# Initialize Faiss index using FAISS.from_texts method


In [ ]:
db = FAISS.from_texts(text_chunks, embedding)
print(db.index.ntotal)

27


### Querying

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

In [ ]:
print(docs[0].page_content)

- Vivekanandar Illam or Vivekananda house is remembered as the place where Swami Vivekananda 
stayed for nine days when he visited Chennai (then Madras) in 1897. Vivekananda House now houses a 
permanent exhibition on Indian Culture. Located on the busy Kamrajar Salai along the Marina Beach, it 
has become an important spiritual tourist attraction in the city.Valluvar Kottam  
- Valluvar Kottam is a popular monument in Chennai, dedicated to the classical Tamil poet, philosopher, 
and saint, Thiruvalluvar who wrote his famous Thirukkural some 2,000 years ago. All 133 chapters and 
1330 verses of the Thirukkural are inscribed on bas-relief in the front hall's corridors. A life-size statue of 
Thiruvalluvar has been installed in the 39 m high chariot.St.Mary's Church inside Fort St.George


### As a Retriever

In [ ]:
retriever = db.as_retriever()
docs = retriever.invoke(query)

In [ ]:
print(docs[0].page_content)

- Vivekanandar Illam or Vivekananda house is remembered as the place where Swami Vivekananda 
stayed for nine days when he visited Chennai (then Madras) in 1897. Vivekananda House now houses a 
permanent exhibition on Indian Culture. Located on the busy Kamrajar Salai along the Marina Beach, it 
has become an important spiritual tourist attraction in the city.Valluvar Kottam  
- Valluvar Kottam is a popular monument in Chennai, dedicated to the classical Tamil poet, philosopher, 
and saint, Thiruvalluvar who wrote his famous Thirukkural some 2,000 years ago. All 133 chapters and 
1330 verses of the Thirukkural are inscribed on bas-relief in the front hall's corridors. A life-size statue of 
Thiruvalluvar has been installed in the 39 m high chariot.St.Mary's Church inside Fort St.George


### Similarity Search with score

In [ ]:
docs_and_scores = db.similarity_search_with_score(query)

In [ ]:
docs_and_scores[0]

(Document(page_content="- Vivekanandar Illam or Vivekananda house is remembered as the place where Swami Vivekananda \nstayed for nine days when he visited Chennai (then Madras) in 1897. Vivekananda House now houses a \npermanent exhibition on Indian Culture. Located on the busy Kamrajar Salai along the Marina Beach, it \nhas become an important spiritual tourist attraction in the city.Valluvar Kottam  \n- Valluvar Kottam is a popular monument in Chennai, dedicated to the classical Tamil poet, philosopher, \nand saint, Thiruvalluvar who wrote his famous Thirukkural some 2,000 years ago. All 133 chapters and \n1330 verses of the Thirukkural are inscribed on bas-relief in the front hall's corridors. A life-size statue of \nThiruvalluvar has been installed in the 39 m high chariot.St.Mary's Church inside Fort St.George"),
 0.5757053)

In [ ]:
embedding_vector = embedding.embed_query(query)
docs_and_scores = db.similarity_search_by_vector(embedding_vector)

In [ ]:
docs_and_scores[0]

Document(page_content="- Vivekanandar Illam or Vivekananda house is remembered as the place where Swami Vivekananda \nstayed for nine days when he visited Chennai (then Madras) in 1897. Vivekananda House now houses a \npermanent exhibition on Indian Culture. Located on the busy Kamrajar Salai along the Marina Beach, it \nhas become an important spiritual tourist attraction in the city.Valluvar Kottam  \n- Valluvar Kottam is a popular monument in Chennai, dedicated to the classical Tamil poet, philosopher, \nand saint, Thiruvalluvar who wrote his famous Thirukkural some 2,000 years ago. All 133 chapters and \n1330 verses of the Thirukkural are inscribed on bas-relief in the front hall's corridors. A life-size statue of \nThiruvalluvar has been installed in the 39 m high chariot.St.Mary's Church inside Fort St.George")

In [ ]:
# Define a function to retrieve the most relevant answers to a question


In [ ]:
# Define a function to retrieve the most relevant answers to a question
def retrieve_answers(question, top_k=3):
    # Embed the question
    question_embedding = embedding.embed(question)  # Corrected method name
    # Search for similar embeddings in Faiss index
    D, I = db.search(np.array([question_embedding]), top_k)
    # Retrieve the corresponding text chunks
    answers = [text_chunks[i[0]] for i in I[0]]
    return answers

In [ ]:
# Ask the user for a question
user_question = "vivekananda"


In [ ]:
# Retrieve the most relevant answers
relevant_answers = retrieve_answers(user_question)

# Print the most relevant answers
print("Most relevant answers to your question:")
for i, answer in enumerate(relevant_answers, 1):
    print(f"{i}. {answer}")

AttributeError: 'OpenAIEmbeddings' object has no attribute 'embed'

### code4

In [10]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.vectorstores import Cassandra
from langchain.embeddings import OpenAIEmbeddings


In [11]:
# Read PDF and extract text
pdf_path = '/content/tourist_attractions_combined.pdf'
pdf_reader = PdfReader(pdf_path)
raw_text = ''
for page in pdf_reader.pages:
    content = page.extract_text()
    if content:
        raw_text += content

In [12]:
# Initialize OpenAI embeddings
OPENAI_API_KEY = "sk-proj-6MJfPIj5KDBPBxzCuxBBT3BlbkFJmL198mgSzQLxuVh1yB3y"
embedding_openai = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [13]:
# Initialize Cassandra vector store for OpenAI embeddings
astra_vector_store_openai = Cassandra(
    embedding=embedding_openai,
    table_name="qa_mini_demo_openai",
    session=None,
    keyspace=None,
)

In [14]:
# Split text into chunks
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)
texts = text_splitter.split_text(raw_text)


In [15]:
# Add texts to Cassandra vector store for OpenAI embeddings
astra_vector_store_openai.add_texts(texts)
print("Inserted %i headlines." % len(texts))

Inserted 151 headlines.


In [16]:
# Initialize VectorStoreIndexWrapper with Cassandra vector store for OpenAI embeddings
astra_vector_index_openai = VectorStoreIndexWrapper(vectorstore=astra_vector_store_openai)

In [17]:
# Initialize llama model
llama_model_path = 'path/to/llama/model'  # Replace with the actual path to your llama model

In [18]:
! pip install -qq -U transformers


In [19]:
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)

In [20]:
import torch
! pip install -qq -U bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.3 MB/s eta 0:00:00


In [21]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_repo = 'daryl149/llama-2-7b-chat-hf'
bnb_config = BitsAndBytesConfig(
            load_in_4bit = True,
            bnb_4bit_quant_type = "nf4",
            bnb_4bit_compute_dtype = torch.float16,
            bnb_4bit_use_double_quant = True,
        )




In [22]:
! pip install -qq -U accelerate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.8 MB/s eta 0:00:00


In [24]:
!pip install accelerate

In [25]:
llm = AutoModelForCausalLM.from_pretrained(
            model_repo,
            quantization_config = bnb_config,
            device_map = 'auto',
            low_cpu_mem_usage = True
        )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install accelerate`

In [ ]:
# Define a function to retrieve the most relevant answers to a question
def retrieve_answers(question, top_k=3):
    # Embed the question using OpenAI embeddings
    question_embedding = embedding_openai.embed(question)
    # Search for similar embeddings in Cassandra vector store for OpenAI embeddings
    D, I = astra_vector_index_openai.search(np.array([question_embedding]), top_k)
    # Retrieve the corresponding text chunks
    answers = [text_chunks[i[0]] for i in I[0]]

    # Use llama model to generate answers based on the retrieved text chunks
    llama_answers = []
    for answer_text in answers:
        llama_answer = llm.generate_answer(question, answer_text)
        llama_answers.append(llama_answer)
    return llama_answers

In [ ]:
# Ask the user for a question
user_question = input("\nEnter your question (or type 'quit' to exit): ").strip()
